In [1]:
! pip install git+https://git@github.com/TheSoundOfAIOSR/rg_text_to_sound.git#"subdirectory=playground/mirco_nani/tts_pipeline" 

  Cloning https://****@github.com/TheSoundOfAIOSR/rg_text_to_sound.git to /tmp/pip-req-build-fwb8rpdh
  Running command git clone -q 'https://****@github.com/TheSoundOfAIOSR/rg_text_to_sound.git' /tmp/pip-req-build-fwb8rpdh
     |████████████████████████████████| 14.5MB 293kB/s 
     |████████████████████████████████| 3.4MB 42.9MB/s 
  Created wheel for soundofai-osr-tts-pipeline: filename=soundofai_osr_tts_pipeline-0.1.0-cp37-none-any.whl size=6471 sha256=fa4c813a14cea54e6d693924d8686b71b8dbe4b7c15bcbcdd603d5b1c7bc6adf
  Stored in directory: /tmp/pip-ephem-wheel-cache-izlocotf/wheels/f7/77/e2/d08680eb4eb9b81c459ab0a93c19b61285f09ebd050554ca36
Successfully built soundofai-osr-tts-pipeline
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:

# Pipeline usage

In [4]:
from tts_pipeline.pipelines.waterfall.pipeline import WaterfallPipeline
from tts_pipeline.pipelines.waterfall.models.examples import (
    DummyWaterfallKeywordExtractor,
    BERTWaterfallEmbedder,
    DummyWaterfallDimensionalityReducer
)

pipeline = WaterfallPipeline(
    keyword_extractor = DummyWaterfallKeywordExtractor(),
    embedder = BERTWaterfallEmbedder(tf_hub_url="https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1"),
    dimensionality_reducer = DummyWaterfallDimensionalityReducer()
)

pipeline.build()
pipeline.predict("a sentence")

{'latent_sample': [0.006180914584547281,
  0.8269093036651611,
  -0.7264376282691956,
  -0.8143622279167175,
  0.7739483714103699,
  -0.030832337215542793,
  0.718966007232666,
  0.976027250289917,
  0.02572336420416832,
  -0.47014039754867554,
  -0.9980174899101257,
  -0.7777827382087708,
  0.9996303915977478,
  0.2778970003128052,
  -0.5174909830093384,
  0.6772952079772949],
 'pitch': 60,
 'qualities': ['a', 'sentence'],
 'source': 'sentence',
 'velocity': 75}

In [5]:
pipeline.predict("this is a longer sentence and it's awesome")

{'latent_sample': [0.04316964000463486,
  0.8932338953018188,
  0.44633564352989197,
  -0.4507346749305725,
  0.9725609421730042,
  0.005000357050448656,
  -0.45076096057891846,
  0.9331349730491638,
  -0.013016034848988056,
  -0.23574545979499817,
  -0.9992653727531433,
  -0.9355983734130859,
  0.9984662532806396,
  0.971244215965271,
  -0.9185348153114319,
  0.8730870485305786],
 'pitch': 60,
 'qualities': ['this',
  'is',
  'a',
  'longer',
  'sentence',
  'and',
  "it's",
  'awesome'],
 'source': 'sentence',
 'velocity': 75}

In [6]:
pipeline.dispose()

In [7]:
model = DummyWaterfallKeywordExtractor()
model.build()
model.predict("my sentence")

{'instrument': 'sentence',
 'pitch': 60,
 'soundquality': ['my', 'sentence'],
 'velocity': 75}

# Cutom model - Embedder

In [15]:
import tensorflow_hub as hub
from tts_pipeline.pipelines.waterfall.pipeline import WaterfallEmbedder

class UniversalSentenceEncoderWaterfallEmbedder(WaterfallEmbedder):
  def __init__(self, tf_hub_url: str = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"):
    self.tf_hub_url = tf_hub_url
    self.built=False

  def build(self):
    self.model = hub.load(self.tf_hub_url)
    self.built=True

  def predict(self, sentence: str, keyword_extraction_results: dict):
    assert self.built, "the model is not built, please invoke the build() method first"
    result = {}
    result["sentence"] = self.model([sentence]).numpy()[0].tolist()
    result["soundquality"] = self.model(keyword_extraction_results["soundquality"]).numpy().tolist()
    result["instrument"] = self.model([keyword_extraction_results["instrument"]]).numpy()[0].tolist()
    return result

  def dispose(self):
    del self.model
    self.built=False

In [ ]:
model = UniversalSentenceEncoderWaterfallEmbedder()
model.build()
model.predict(
    "this is a sentence",
    {
        "soundquality": ["bright", "clean"],
        "instrument": "acoustic guitar",
        "pitch": 10,
        "velocity": 10
    }
)

In [17]:
result = model.predict(
    "this is a sentence",
    {
        "soundquality": ["bright", "clean"],
        "instrument": "acoustic guitar",
        "pitch": 10,
        "velocity": 10
    }
)

In [18]:
result.keys()

dict_keys(['sentence', 'soundquality', 'instrument'])

In [19]:
len(result["sentence"])

512

In [20]:
result["sentence"][:32]

[0.028817646205425262,
 -0.020200153812766075,
 0.01069628819823265,
 0.038505300879478455,
 -0.09253698587417603,
 0.017527736723423004,
 -0.047117531299591064,
 0.04785209894180298,
 0.014307161793112755,
 0.026359528303146362,
 0.02157277800142765,
 -0.04987606406211853,
 0.030238166451454163,
 0.1192370057106018,
 0.04333100840449333,
 -0.11343763023614883,
 -0.025693802163004875,
 0.008759822696447372,
 -0.03205025568604469,
 -0.035288915038108826,
 0.06509800255298615,
 0.07168703526258469,
 0.00487420242279768,
 -0.006229517515748739,
 -0.054833851754665375,
 0.06636364758014679,
 0.021976977586746216,
 -0.08987361192703247,
 0.036639127880334854,
 -0.041861094534397125,
 0.030207039788365364,
 -0.021765589714050293]

In [22]:
model.dispose()

In [23]:
pipeline = WaterfallPipeline(
    keyword_extractor = DummyWaterfallKeywordExtractor(),
    embedder = UniversalSentenceEncoderWaterfallEmbedder(),
    dimensionality_reducer = DummyWaterfallDimensionalityReducer()
)

pipeline.build()
pipeline.predict("a sentence")

{'latent_sample': [-0.023791885003447533,
  -0.02070368081331253,
  0.03718067705631256,
  -0.014570570550858974,
  -0.08293148875236511,
  0.06481515616178513,
  -0.053779907524585724,
  -0.0019122824305668473,
  -7.766970520606264e-05,
  -0.0022318423725664616,
  0.020326251164078712,
  -0.0382310152053833,
  0.033373527228832245,
  0.10438230633735657,
  0.012718302197754383,
  -0.10266664624214172],
 'pitch': 60,
 'qualities': ['a', 'sentence'],
 'source': 'sentence',
 'velocity': 75}

In [24]:
pipeline.dispose()